In [1]:
!apt-get -y install poppler-utils tesseract-ocr libtesseract-dev tesseract-ocr-rus

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libtesseract-dev is already the newest version (4.1.1-2.1build1).
tesseract-ocr is already the newest version (4.1.1-2.1build1).
tesseract-ocr-rus is already the newest version (1:4.00~git30-7274cfa-1.1).
poppler-utils is already the newest version (22.02.0-2ubuntu0.6).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [2]:
!pip uninstall -y gradio

Found existing installation: gradio 3.50.2
Uninstalling gradio-3.50.2:
  Successfully uninstalled gradio-3.50.2


In [3]:
# Cell 3 (Modified Again)
!pip install -q numpy ctransformers[cuda] sentence-transformers chromadb langchain langchain-community langchain-huggingface gradio==3.50.2 unstructured unstructured[pdf] unstructured[docx] python-docx fastapi uvicorn[standard] nest_asyncio pyngrok --quiet

In [4]:
# Install ctransformers with CPU support first (more reliable than GPU on some configurations)
!pip install -q ctransformers --quiet

In [5]:
import os
import torch
import gradio as gr
import requests
from ctransformers import AutoModelForCausalLM
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings

# --- New API Imports ---
import uvicorn
from fastapi import FastAPI, HTTPException, Request
from fastapi.responses import JSONResponse
from pydantic import BaseModel
# --- End New API Imports ---

In [6]:
# Check GPU availability
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

CUDA available: True
CUDA device: Tesla T4


In [7]:
# Model settings
MODEL_URL = "https://huggingface.co/blues-alex/YandexGPT-5-Lite-8B-pretrain-Q4_K_M-GGUF/resolve/main/yandexgpt-5-lite-8b-pretrain-q4_k_m.gguf"
MODEL_PATH = "./yandexgpt-5-lite-8b-pretrain-q4_k_m.gguf"
RAG_DIR = "/content/rag_db"
CURRENT_CONTEXT_SIZE = 2048  # Default context size
CURRENT_GPU_LAYERS = 20      # Default GPU layers

In [8]:
# Function to download model
def download_model(url, save_path):
    if os.path.exists(save_path):
        return "Модель уже загружена"
    try:
        print(f"Загрузка модели из {url}...")
        response = requests.get(url, stream=True)
        response.raise_for_status()
        total_size = int(response.headers.get('content-length', 0))
        block_size = 8192
        downloaded = 0

        with open(save_path, "wb") as f:
            for chunk in response.iter_content(chunk_size=block_size):
                f.write(chunk)
                downloaded += len(chunk)
                if total_size > 0:
                    print(f"\rЗагружено: {downloaded/1024/1024:.1f}MB / {total_size/1024/1024:.1f}MB ({downloaded*100/total_size:.1f}%)", end="")

        print("\nМодель успешно загружена")
        return "Модель успешно загружена"
    except requests.RequestException as e:
        return f"Ошибка загрузки модели: {e}"

In [9]:
# Download model if needed
if not os.path.exists(MODEL_PATH):
    print("Скачивание модели...")
    download_model(MODEL_URL, MODEL_PATH)

In [10]:
# Добавьте эту функцию перед функциями update_model и change_context_size

def load_model_with_params(context_size=CURRENT_CONTEXT_SIZE, gpu_layers=CURRENT_GPU_LAYERS):
    """
    Загружает модель с указанными параметрами размера контекста и количества GPU слоев.

    Args:
        context_size (int): Размер контекстного окна модели
        gpu_layers (int): Количество слоев для выполнения на GPU

    Returns:
        tuple: (model, mode) - модель и режим работы (GPU/CPU)
    """
    from ctransformers import AutoModelForCausalLM

    try:
        # Try with GPU support
        print(f"Загрузка модели с контекстом {context_size} и {gpu_layers} GPU слоями...")
        model = AutoModelForCausalLM.from_pretrained(
            MODEL_PATH,
            model_type="llama",
            gpu_layers=gpu_layers,
            context_length=context_size,
            batch_size=512
        )
        print(f"Модель успешно загружена с контекстом {context_size}")
        return model, "GPU"
    except Exception as e:
        print(f"Ошибка при загрузке модели с GPU: {e}")
        print("Загружаем модель в режиме CPU...")

        try:
            model = AutoModelForCausalLM.from_pretrained(
                MODEL_PATH,
                model_type="llama",
                gpu_layers=0,  # CPU only mode
                context_length=context_size,
                batch_size=512
            )
            print(f"Модель загружена в режиме CPU с контекстом {context_size}")
            return model, "CPU"
        except Exception as e:
            raise Exception(f"Не удалось загрузить модель: {e}")

In [11]:
# Create directories
os.makedirs("/content/Doc", exist_ok=True)

In [12]:
# Global variable to hold the sample document creation function
def create_sample_document():
    """Create a sample document for RAG testing if none exists"""
    sample_path = "/content/Doc/sample.docx"
    try:
        from docx import Document
        doc = Document()
        doc.add_paragraph("Этот документ создан для примера работы системы RAG с УрФУ.")
        doc.add_paragraph("Уральский федеральный университет (УрФУ) расположен в Екатеринбурге.")
        doc.add_paragraph("УрФУ является одним из ведущих вузов России.")
        doc.add_paragraph("В УрФУ обучаются студенты со всей России и из многих зарубежных стран.")
        doc.add_paragraph("УрФУ предлагает программы бакалавриата, магистратуры и аспирантуры.")
        doc.save(sample_path)
        print(f"✅ Создан пример документа для тестирования: {sample_path}")
        return True
    except Exception as e:
        print(f"❌ Ошибка при создании примера документа: {e}")
        return False

# RAG functions with improved error handling
def initialize_rag():
    """Initialize the RAG database with documents"""
    try:
        # Ensure the directory exists
        os.makedirs("/content/Doc", exist_ok=True)

        # Check if there are any documents
        doc_files = [f for f in os.listdir("/content/Doc") if f.endswith(".docx")]

        if not doc_files:
            print("Нет документов для загрузки. Создаем пример документа...")
            created = create_sample_document()
            if not created:
                print("Не удалось создать пример документа. Инициализация RAG не выполнена.")
                return None

        # Load documents
        loader = DirectoryLoader("/content/Doc", glob="**/*.docx")
        documents = loader.load()

        if not documents:
            print("Не удалось загрузить документы, даже после создания примера.")
            print("Проверьте, что папка /content/Doc содержит доступные файлы .docx")
            return None

        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        texts = text_splitter.split_documents(documents)

        if not texts:
            print("Документы загружены, но не удалось извлечь текст.")
            return None

        embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
        vector_db = Chroma.from_documents(texts, embeddings, persist_directory=RAG_DIR)
        print(f"✅ База RAG успешно инициализирована с {len(texts)} фрагментами")
        return vector_db

    except Exception as e:
        print(f"❌ Ошибка инициализации RAG: {e}")
        return None


In [13]:
# Load or initialize RAG database with better error handling
vector_db = None  # Initialize to None first
try:
    if not os.path.exists(RAG_DIR):
        print("RAG база данных не найдена, создаем новую...")
        vector_db = initialize_rag()
        if vector_db is None:
            print("❌ Не удалось инициализировать RAG. Ответы модели не будут использовать контекст документов.")
    else:
        try:
            print("Загружаем существующую базу RAG...")
            embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
            vector_db = Chroma(persist_directory=RAG_DIR, embedding_function=embeddings)
            print("✅ База RAG успешно загружена")
        except Exception as e:
            print(f"❌ Ошибка загрузки существующей базы RAG: {e}")
            print("Пробуем создать новую базу RAG...")
            if os.path.exists(RAG_DIR):
                import shutil
                shutil.rmtree(RAG_DIR)
            vector_db = initialize_rag()
            if vector_db is None:
                print("❌ Не удалось инициализировать RAG. Ответы модели не будут использовать контекст документов.")
except Exception as e:
    print(f"❌ Неожиданная ошибка при работе с RAG: {e}")

Загружаем существующую базу RAG...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
<ipython-input-13-9098590e027e>:13: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_db = Chroma(persist_directory=RAG_DIR, embedding_function=embeddings)


✅ База RAG успешно загружена


In [14]:
# Фиксированный контекст, который будет передаваться модели
FIXED_CONTEXT = """Ты — интеллектуальный помощник, обученный отвечать на вопросы строго в рамках предоставленного контекста. Ты помощник по Уральскому Федеральному университету, который общается с пользователем. Общайся с ним только на русском языке. Объясняй максимально подробно, чтобы пользователь всё понял. Если информации недостаточно, отвечай "Вы можете ознакомиться с данной информацией на сайте УрФУ". Избегай выдумок и предположений. Если пользователь просит дать точные значения - обращайся только к данным RAG."""
CURRENT_TEMPERATURE = 0.5  # Значение по умолчанию
# Function to generate response using RAG
# Обновленная функция для генерации ответов с улучшенным контролем источников
def ask_question_with_rag(question):
    try:
        # Check if vector_db is available
        if vector_db is None:
            # Fallback to model-only generation
            prompt = f"{FIXED_CONTEXT}\n\nUser: {question}\nAssistant:"
            response = model(
                prompt,
                max_new_tokens=256,
                temperature=CURRENT_TEMPERATURE,
                stop=["User:", "\n\n"]
            )
            return response.strip() + "\n\n(Примечание: ответ дан без использования базы знаний, так как RAG не инициализирован)"

        # Retrieve relevant documents
        docs = vector_db.similarity_search(question, k=3)

        # Check if the retrieved documents are actually relevant by looking for keywords
        query_keywords = set(question.lower().split())
        important_keywords = {word for word in query_keywords
                             if len(word) > 3 and word not in
                             ['что', 'как', 'где', 'когда', 'какие', 'какой', 'какая', 'институт', 'урфу', 'университет', 'ИРИТ-РТФ', 'РТФ']}

        # Extract and show relevant context
        extracted_context = "\n\n".join([doc.page_content for doc in docs])

        # Check if any important keywords are in the context
        found_keywords = []
        for keyword in important_keywords:
            if keyword in extracted_context.lower():
                found_keywords.append(keyword)

        # Determine relevance score
        relevance_score = len(found_keywords) / max(1, len(important_keywords)) if important_keywords else 0.5

        # Add strong instruction about only using provided context
        strict_instruction = """
ВАЖНО: Отвечай ТОЛЬКО на основе предоставленной информации из документов.
Если в предоставленных документах нет ответа на вопрос, честно скажи
"В документах нет информации о [тема вопроса]. Вы можете ознакомиться с этой информацией на сайте УрФУ."
НЕ ПРИДУМЫВАЙ информацию, которой нет в документах!
"""
        combined_context = f"{FIXED_CONTEXT}\n\n{strict_instruction}\n\nДокументы:\n{extracted_context}"

        # Create prompt for model with stronger guidance
        prompt = f"Context: {combined_context}\n\nUser: {question}\n\nAssistant:"

        # Use even lower temperature for low relevance scores to reduce hallucination
        adjusted_temperature = min(CURRENT_TEMPERATURE, 0.3) if relevance_score < 0.5 else CURRENT_TEMPERATURE

        # Generate response with ctransformers
        response = model(
            prompt,
            max_new_tokens=256,
            temperature=adjusted_temperature,  # Use adjusted temperature based on relevance
            stop=["User:", "\n\n"]
        )

        # Return the generated text with optional debugging info
        result = response.strip()

        # For debugging - uncomment to show relevance information
        # debug_info = f"\n\n[Отладка: Найдено {len(found_keywords)}/{len(important_keywords)} ключевых слов, релевантность {relevance_score:.2f}]"
        # return result + debug_info

        return result

    except Exception as e:
        return f"Произошла ошибка при генерации ответа: {str(e)}\n\nПожалуйста, попробуйте очистить и переинициализировать RAG."

# Function to update model
def update_model(link):
    try:
        global model, mode
        result = download_model(link, MODEL_PATH)
        # Reload with current context size
        model, mode = load_model_with_params(CURRENT_CONTEXT_SIZE, CURRENT_GPU_LAYERS)
        return f"Модель обновлена: {result}. Режим работы: {mode}"
    except Exception as e:
        return f"Ошибка при обновлении модели: {str(e)}"

# Function to change context size
def change_context_size(new_size_str):
    try:
        global model, CURRENT_CONTEXT_SIZE, mode

        # Convert to integer and validate
        new_size = int(new_size_str)
        if new_size < 512:
            return "Ошибка: размер контекста должен быть не менее 512"
        if new_size > 8192:
            return "Ошибка: размер контекста не может превышать 8192"

        CURRENT_CONTEXT_SIZE = new_size
        model, mode = load_model_with_params(CURRENT_CONTEXT_SIZE, CURRENT_GPU_LAYERS)
        return f"Размер контекста изменен на {new_size}. Режим работы: {mode}"
    except ValueError:
        return "Ошибка: введите корректное целое число"
    except Exception as e:
        return f"Ошибка при изменении размера контекста: {str(e)}"

def change_temperature(new_temp_str):
    try:
        global CURRENT_TEMPERATURE

        # Convert to float and validate
        new_temp = float(new_temp_str)
        if new_temp < 0.0:
            return "Ошибка: температура не может быть меньше 0.0"
        if new_temp > 2.0:
            return "Ошибка: температура не рекомендуется выше 2.0"

        CURRENT_TEMPERATURE = new_temp
        return f"Температура генерации изменена на {new_temp}"
    except ValueError:
        return "Ошибка: введите корректное число с плавающей точкой (например, 0.2)"
    except Exception as e:
        return f"Ошибка при изменении температуры: {str(e)}"

# Function to clear RAG
def clear_rag():
    try:
        if os.path.exists(RAG_DIR):
            import shutil
            shutil.rmtree(RAG_DIR)
        global vector_db
        vector_db = initialize_rag()
        return "RAG очищен и переинициализирован"
    except Exception as e:
        return f"Ошибка при очистке RAG: {str(e)}"

In [15]:
# Cell 14.1 (New) - API Models
class QuestionRequest(BaseModel):
    question: str

class AnswerResponse(BaseModel):
    answer: str

In [16]:
# Cell 14.5 (New) - Install cloudflared
print("Установка cloudflared...")
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared
print("cloudflared установлен.")
# Переместим в /usr/local/bin для удобства вызова (не обязательно, но рекомендуется)
!mv cloudflared /usr/local/bin/

Установка cloudflared...
cloudflared установлен.


In [ ]:
# Cell 15 (Возвращена Админ-Панель, БЕЗ Whisper, с Cloudflared)

import asyncio
import nest_asyncio
import uvicorn
import os
import subprocess
import re
import time
from pydantic import BaseModel # Убедимся, что импортирован
from fastapi import FastAPI, HTTPException # Убедимся, что импортированы
import gradio as gr # Убедимся, что импортирован

# --- API Models (если еще не определены выше) ---
class QuestionRequest(BaseModel):
    question: str

class AnswerResponse(BaseModel):
    answer: str

# --- 1. Load Model and Initialize RAG (as before) ---
print("Загрузка модели...")
model, mode = load_model_with_params(CURRENT_CONTEXT_SIZE, CURRENT_GPU_LAYERS)
print(f"Модель загружена. Режим: {mode}")
# Убедитесь, что RAG инициализирован (vector_db существует)

# --- 2. Define Gradio UI (Чат + ВОЗВРАЩЕННАЯ АДМИН-ПАНЕЛЬ) ---
print("Определение интерфейса Gradio (Чат + Админ-панель)...")
with gr.Blocks() as demo:
    # --- Чат ---
    with gr.Row():
        gr.Markdown("### Чат с AI")
    chat_input = gr.Textbox(label="Введите вопрос")
    chat_output = gr.Textbox(label="Ответ", lines=10, interactive=False) # Output non-interactive
    submit_button = gr.Button("Отправить")
    # Связываем кнопку отправки чата
    submit_button.click(ask_question_with_rag, inputs=chat_input, outputs=chat_output)

    # --- ВОЗВРАЩЕННАЯ АДМИН-ПАНЕЛЬ ---
    with gr.Row():
        gr.Markdown("### Админ-панель")

    with gr.Row():
        # Колонка обновления модели
        with gr.Column():
            model_link = gr.Textbox(label="Ссылка на модель (GGUF)")
            update_model_button = gr.Button("Обновить модель")
            update_model_output = gr.Textbox(label="Статус обновления модели", interactive=False)
            # Связываем кнопку обновления модели
            update_model_button.click(update_model, inputs=model_link, outputs=update_model_output)

        # Колонка изменения размера контекста
        with gr.Column():
            context_size_input = gr.Textbox(label="Размер контекста (512-8192)", value=str(CURRENT_CONTEXT_SIZE))
            context_size_button = gr.Button("Изменить размер контекста")
            context_size_output = gr.Textbox(label="Статус изменения контекста", interactive=False)
            # Связываем кнопку изменения контекста
            context_size_button.click(change_context_size, inputs=context_size_input, outputs=context_size_output)

    with gr.Row():
        # Колонка изменения температуры
        with gr.Column():
            temperature_input = gr.Textbox(
                label="Температура (0.0-2.0)",
                value=str(CURRENT_TEMPERATURE)
            )
            temperature_button = gr.Button("Изменить температуру")
            temperature_output = gr.Textbox(label="Статус изменения температуры", interactive=False)
            # Связываем кнопку изменения температуры
            temperature_button.click(change_temperature, inputs=temperature_input, outputs=temperature_output)

        # Колонка очистки RAG
        with gr.Column():
            rag_button = gr.Button("Очистить и переинициализировать RAG")
            rag_output = gr.Textbox(label="Статус RAG", interactive=False)
            # Связываем кнопку очистки RAG
            rag_button.click(clear_rag, outputs=rag_output)

print("Интерфейс Gradio определен.")

# --- 3. Create FastAPI App ---
print("Создание FastAPI приложения...")
api_app = FastAPI(title="Text Generation API", description="API для генерации текста с использованием RAG")
print("FastAPI приложение создано.")

# --- 4. Define API Endpoint (/api/ask) ---
@api_app.post("/api/ask",
              response_model=AnswerResponse,
              summary="Задать вопрос модели",
              description="Отправляет вопрос модели (с использованием RAG, если доступно) и возвращает ответ.")
async def handle_ask_api(request: QuestionRequest):
    print(f"[API Request] /api/ask - Вопрос: {request.question}")
    try:
        answer = ask_question_with_rag(request.question)
        print(f"[API Response] /api/ask - Ответ сгенерирован.")
        return AnswerResponse(answer=answer)
    except Exception as e:
        print(f"[API Error] /api/ask - Ошибка: {str(e)}")
        import traceback
        traceback.print_exc()
        raise HTTPException(status_code=500, detail=f"Ошибка генерации ответа: {str(e)}")
print("API эндпоинт /api/ask определен.")

# --- 5. Mount Gradio App onto FastAPI ---
print("Монтирование Gradio UI на FastAPI...")
# Убедитесь, что 'demo' - это ваш объект gr.Blocks()
api_app = gr.mount_gradio_app(api_app, demo, path="/")
print("Gradio UI смонтирован на FastAPI по пути '/'.")

# --- 6. Run with Uvicorn and Cloudflared Tunnel ---
nest_asyncio.apply()

async def run_server_and_tunnel():
    config = uvicorn.Config(app=api_app, host="0.0.0.0", port=7860, log_level="info")
    server = uvicorn.Server(config)
    server_task = asyncio.create_task(server.serve())
    print("Сервер Uvicorn запущен в фоновом режиме.")
    await asyncio.sleep(5)

    print("Запуск cloudflared...")
    cf_process = subprocess.Popen(
        ['cloudflared', 'tunnel', '--url', 'http://localhost:7860'],
        stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True
    )

    public_url = None
    login_url_printed = False
    start_time = time.time()
    try:
        while time.time() - start_time < 60:
            line = cf_process.stderr.readline()
            if not line and cf_process.poll() is not None: break
            if not line:
                await asyncio.sleep(0.5); continue
            print(f"[cloudflared] {line.strip()}")

            # Логин URL (первый раз)
            if "https://dash.cloudflare.com/argotunnel?callback=" in line:
                 login_url = re.search(r'(https://dash.cloudflare.com/argotunnel\?callback=[^\s]+)', line)
                 if login_url:
                      print("\n" + "="*50 + "\n‼️ ТРЕБУЕТСЯ АУТЕНТИФИКАЦИЯ CLOUDFLARE (ОДИН РАЗ) ‼️")
                      print(f"1. Скопируйте ссылку:\n    {login_url.group(1)}\n")
                      print("2. Откройте в браузере, войдите в Cloudflare, авторизуйте.")
                      print("3. После успеха в браузере, ОСТАНОВИТЕ и ПЕРЕЗАПУСТИТЕ эту ячейку.\n" + "="*50 + "\n")
                      login_url_printed = True; cf_process.terminate(); await cf_process.wait(); return

            # Готовый URL (после логина)
            tunnel_url_match = re.search(r'(https://[a-zA-Z0-9-]+\.trycloudflare\.com)', line)
            if tunnel_url_match:
                public_url = tunnel_url_match.group(1)
                print("\n" + "="*50)
                print(f"✅ Публичный URL (Cloudflare Tunnel): {public_url}")
                print(f"   API: {public_url}/api/ask | Docs: {public_url}/docs")
                print("--- Сервер и туннель активны. Остановите ячейку для завершения. ---\n" + "="*50 + "\n")
                break # URL найден

        if not public_url and not login_url_printed:
             print("Не удалось получить URL от cloudflared за 60 секунд."); cf_process.terminate(); await cf_process.wait(); return

        await server_task # Ждем завершения Uvicorn

    except Exception as e:
        print(f"❌ Ошибка: {e}"); server_task.cancel()
        try: await server_task
        except asyncio.CancelledError: print("Сервер Uvicorn остановлен.")
    finally:
        print("Завершение cloudflared...");
        if cf_process.poll() is None: # Проверяем, жив ли еще процесс
             cf_process.terminate()
             try:
                 await asyncio.wait_for(asyncio.to_thread(cf_process.wait), timeout=5.0)
             except asyncio.TimeoutError:
                 print("Процесс cloudflared не завершился вовремя, убиваем...")
                 cf_process.kill()
                 await asyncio.to_thread(cf_process.wait)
        print("Процесс cloudflared завершен.")


# --- Запуск ---
print("Запуск асинхронной функции для сервера и туннеля...")
asyncio.run(run_server_and_tunnel())

Загрузка модели...
Загрузка модели с контекстом 2048 и 20 GPU слоями...
Модель успешно загружена с контекстом 2048
Модель загружена. Режим: GPU
Определение интерфейса Gradio (Чат + Админ-панель)...
Интерфейс Gradio определен.
Создание FastAPI приложения...
FastAPI приложение создано.
API эндпоинт /api/ask определен.
Монтирование Gradio UI на FastAPI...
Gradio UI смонтирован на FastAPI по пути '/'.
Запуск асинхронной функции для сервера и туннеля...


INFO:     Started server process [4977]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:7860 (Press CTRL+C to quit)


Сервер Uvicorn запущен в фоновом режиме.
IMPORTANT: You are using gradio version 3.50.2, however version 4.44.1 is available, please upgrade.
--------
Запуск cloudflared...
[cloudflared] 2025-03-26T10:37:19Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
[cloudflared] 2025-03-26T10:37:19Z INF Requesting new quick Tunnel on trycloudflare.com...
[cloudflared] 2025-03-26T10:37:23Z INF +--------------------------------------------------------------------------------